In [1]:
#'type': This is a string,
#'metadata': dictionary that holds info regarding to this dataset.
'''
  {'generated': 1581310393000,
   'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-01-01&endtime=2017-02-28',
   'title': 'USGS Earthquakes',
   'status': 200,
   'api': '1.8.1',
   'count': 19628}
'''

#'features': This one holds data
#'bbox': Not sure what it is but not useful by looking at it
'''
  [-179.9737, -62.6993, -3.44, 179.9865, 86.1559, 662.16]
'''

Out[1]: '\n [-179.9737, -62.6993, -3.44, 179.9865, 86.1559, 662.16]\n'

In [2]:
#To import necessary modules and write functions
import requests
import json
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
import calendar

def import_range(yr, mth):
  return str(yr)+'-'+(str(mth).zfill(2))+'-'+'01' , str(yr)+'-'+(str(mth).zfill(2))+'-'+str(calendar.monthrange(yr,mth)[1])

def query_df (startdate, enddate):
  url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=' + str(startdate) + '&endtime=' + str(enddate)
  response = requests.get(url)
  earthquake_json = response.json()
  data = [quake['properties'] for quake in earthquake_json['features']]
  df = pd.DataFrame(data)
  return df

In [3]:
#########################################################################
#1) Please query all events that have occurred during year 2017
#2) Read a JSON response from the API
##########################################################################




#create an empty dataframe first
quake_df = pd.DataFrame(columns=['alert', 'cdi', 'code', 'detail', 'dmin', 'felt', 'gap', 'ids', 'mag','magType', 'mmi', 'net', 'nst', 'place', 'rms', 'sig', 'sources','status', 'time', 'title', 'tsunami', 'type', 'types', 'tz', 'updated','url'])

#to query all data for each month since query one year worth of data is not supported at the moment.
for i in range(1, 13):
    start = import_range(2017, i)[0]
    end = import_range(2017, i)[1]
    df = query_df (start, end)
    quake_df = pd.concat([quake_df, df])
    print("number of rows " + str(len(quake_df)))

#convert ms time to datetime
quake_df['time'] = pd.to_datetime(quake_df['time'],unit='ms')

number of rows 11247
number of rows 19292
number of rows 28459
number of rows 39155
number of rows 52789
number of rows 64161
number of rows 77233
number of rows 89533
number of rows 101086
number of rows 110957
number of rows 120829
number of rows 130431

In [4]:
#convert pandas to spark
spark_df = sqlContext.createDataFrame(quake_df)

In [5]:
#register temp table
spark_df.registerTempTable("df_sql")

In [6]:
#####################################################################################################
#3) Design the database objects required to store the result in a relational fashion.
#4) Store the response in those objects
#####################################################################################################

In [7]:
#Below in Cmd8 is the testing to see if all fields selected below are unique so we could create a dim table out of it.

In [8]:
%sql
select
 
  alert,
  code,
  detail,
  ids,
  magType,
  net,
  nst,
  place,
  sources,
  status,
  title,
  type,
  types,
  url,
  count(*)
from df_sql
where alert is not null
group by alert, code, detail, ids, magType, net, nst, place, sources, status,title,type,types,url
order by count(*) desc
limit(10)

alert,code,detail,ids,magType,net,nst,place,sources,status,title,type,types,url,count(1)
green,10008h1u,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us10008h1u&format=geojson,",us10008h1u,",mww,us,null,"9km S of Adtugan, Philippines",",us,",reviewed,"M 5.5 - 9km S of Adtugan, Philippines",earthquake,",dyfi,geoserve,losspager,moment-tensor,origin,phase-data,shakemap,",https://earthquake.usgs.gov/earthquakes/eventpage/us10008h1u,1
green,10007rjn,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us10007rjn&format=geojson,",us10007rjn,",mww,us,null,"54km SW of Jahrom, Iran",",us,",reviewed,"M 5.0 - 54km SW of Jahrom, Iran",earthquake,",geoserve,losspager,moment-tensor,origin,phase-data,shakemap,",https://earthquake.usgs.gov/earthquakes/eventpage/us10007rjn,1
green,10008e3k,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us10008e3k&format=geojson,",us10008e3k,",mww,us,null,"132km WSW of Moijabana, Botswana",",us,",reviewed,"M 6.5 - 132km WSW of Moijabana, Botswana",earthquake,",dyfi,general-text,geoserve,ground-failure,losspager,moment-tensor,origin,phase-data,shakemap,",https://earthquake.usgs.gov/earthquakes/eventpage/us10008e3k,1
green,1000991f,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us1000991f&format=geojson,",us1000991f,",mww,us,null,"77km W of Constitucion, Chile",",us,",reviewed,"M 5.6 - 77km W of Constitucion, Chile",earthquake,",dyfi,geoserve,losspager,moment-tensor,origin,phase-data,shakemap,",https://earthquake.usgs.gov/earthquakes/eventpage/us1000991f,1
green,2000bldw,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us2000bldw&format=geojson,",us2000bldw,",mww,us,null,West Chile Rise,",us,",reviewed,M 5.8 - West Chile Rise,earthquake,",geoserve,losspager,moment-tensor,origin,phase-data,shakemap,",https://earthquake.usgs.gov/earthquakes/eventpage/us2000bldw,1
green,20008l41,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us20008l41&format=geojson,",us20008l41,",mww,us,null,"41km E of Padilla, Bolivia",",us,",reviewed,"M 6.5 - 41km E of Padilla, Bolivia",earthquake,",dyfi,geoserve,losspager,moment-tensor,origin,phase-data,shakemap,",https://earthquake.usgs.gov/earthquakes/eventpage/us20008l41,1
green,1000885n,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us1000885n&format=geojson,",us1000885n,",mww,us,null,"56km N of La Vela de Coro, Venezuela",",us,",reviewed,"M 5.2 - 56km N of La Vela de Coro, Venezuela",earthquake,",dyfi,geoserve,losspager,moment-tensor,origin,phase-data,shakemap,",https://earthquake.usgs.gov/earthquakes/eventpage/us1000885n,1
green,10008qsb,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us10008qsb&format=geojson,",us10008qsb,pt17129002,at00opouj1,",mww,us,null,"62km NE of Port-Olry, Vanuatu",",us,pt,at,",reviewed,"M 6.8 - 62km NE of Port-Olry, Vanuatu",earthquake,",dyfi,geoserve,impact-link,losspager,moment-tensor,origin,phase-data,shakemap,",https://earthquake.usgs.gov/earthquakes/eventpage/us10008qsb,1
green,2000aepg,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us2000aepg&format=geojson,",us2000aepg,uu60229432,",mb,us,null,"10km ESE of Soda Springs, Idaho",",us,uu,",reviewed,"M 4.1 - 10km ESE of Soda Springs, Idaho",earthquake,",dyfi,geoserve,losspager,origin,phase-data,shakemap,",https://earthquake.usgs.gov/earthquakes/eventpage/us2000aepg,1
green,2000c2xt,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us2000c2xt&format=geojson,",us2000c2xt,",mww,us,null,"111km WNW of Iquique, Chile",",us,",reviewed,"M 5.5 - 111km WNW of Iquique, Chile",earthquake,",dyfi,geoserve,losspager,moment-tensor,origin,phase-data,shakemap,",https://earthquake.usgs.gov/earthquakes/eventpage/us2000c2xt,1


In [9]:
%sql
--assuming the tables are in SQL server already, here is how we split the data to dim and fact tables.

CREATE TABLE Dim_EarthQuakeInfo (
  Earthquakeid int IDENTITY(1,1),
  alert nvarchar(500),
  code nvarchar(500),
  detail nvarchar(500),
  ids nvarchar(500),
  magType nvarchar(500),
  net nvarchar(500),
  nst nvarchar(500),
  place nvarchar(500),
  sources nvarchar(500),
  status nvarchar(500),
  title nvarchar(500),
  type nvarchar(500),
  types nvarchar(500),
  url nvarchar(500)
  CONSTRAINT PK_EarthQuakeId PRIMARY KEY (Earthquakeid)
);


--Merge Statement

--Dim Table
MERGE Dim_EarthQuakeInfo as target_table USING 
df_sql as source_table
ON （source_table.ids = target_table.ids）
WHEN MATCHED THEN 
     UPDATE SET
      alert = source_table.alert,
      code  = source_table.code,
      detail  = source_table.detail,
      magType  = source_table.magType,
      net  = source_table.net,
      nst  = source_table.nst,
      place  = source_table.place,
      sources  = source_table.sources,
      status  = source_table.status,
      title  = source_table.title,
      types  = source_table.types,
      type  = source_table.type,
      url  = source_table.url
WHEN NOT MATCHED THEN
INSERT (
        alert,
        code,
        detail,
        magType,
        net,
        nst,
        place,
        sources,
        status,
        title,
        types,
        type,
        url
        )
      VALUES (
      source_table.alert,
       source_table.code,
       source_table.detail,
       source_table.magType,
       source_table.net,
       source_table.nst,
       source_table.place,
       source_table.sources,
       source_table.status,
       source_table.title,
       source_table.types,
       source_table.type,
       source_table.url
        );
END


--Fact Table
CREATE TABLE Fact_EarthQuake (
  Earthquakeid int,
  mag float,
  time timestamp,
  updated timestamp,
  tz float,
  felt float,
  cdi int,
  mmi int,
  tsunami boolean,
  sig int,
  dmin float,
  rms float,
  gap float,
  CONSTRAINT PK_FactEarthQuake PRIMARY KEY (Earthquakeid)
);

MERGE Fact_EarthQuake as target_table USING 
(select *, Earthquakeid from df_sql left join Dim_EarthQuakeInfo de on de.Ids = df_sql.Ids) as source_table
ON （source_table.ids = target_table.ids）
WHEN MATCHED THEN 
     UPDATE SET
      Earthquakeid = source_table.Earthquakeid,
      mag = source_table.mag,
      time = source_table.time,
      updated = source_table.updated,
      tz = source_table.tz,
      felt = source_table.felt,
      cdi = source_table.cdi,
      mmi = source_table.mmi,
      tsunami = source_table.tsunami,
      sig = source_table.sig,
      dmin = source_table.dmin, 
      rms = source_table.rms,
      gap = source_table.gap
WHEN NOT MATCHED THEN
INSERT (
        Earthquakeid,
        mag,
        time,
        updated,
        tz,
        felt,
        cdi,
        mmi,
        tsunami,
        sig,
        dmin, 
        rms,
        gap 
        )
      VALUES (
      source_table.Earthquakeid,
      source_table.mag,
      source_table.time,
      source_table.updated,
      source_table.tz,
      source_table.felt,
      source_table.cdi,
      source_table.mmi,
      source_table.tsunami,
      source_table.sig,
      source_table.dmin,
      source_table.rms,
      source_table.gap
        );
END

In [10]:
##################################################################
#5) Provide query/analysis to give biggest earthquake of 2017
##################################################################

#结论：最大的地震是发生早2017年09月08日的墨西哥

In [11]:
%sql
select
  place,
  time,
  max(mag)
from df_sql
where type = 'earthquake'
group by place,time
order by max(mag) desc


place,time,max(mag)
"101km SSW of Tres Picos, Mexico",2017-09-08T04:49:19.180+0000,8.2
"35km WNW of Panguna, Papua New Guinea",2017-01-22T04:30:22.960+0000,7.9
"202km ESE of Nikol'skoye, Russia",2017-07-17T23:34:13.740+0000,7.7
"29km S of Halabjah, Iraq",2017-11-12T18:18:17.180+0000,7.3
"189km SSE of Tabiauan, Philippines",2017-01-10T06:13:48.140+0000,7.3
"1km E of Ayutla, Mexico",2017-09-19T18:14:38.090+0000,7.1
"85km ENE of Tadine, New Caledonia",2017-11-19T22:43:29.250+0000,7.0
"40km W of Valparaiso, Chile",2017-04-24T21:38:30.820+0000,6.9
"31km SW of Burias, Philippines",2017-04-28T20:23:17.260+0000,6.9
"289km S of Ndoi Island, Fiji",2017-02-24T17:28:44.840+0000,6.9


In [12]:
#######################################################################################################
#6) Provide query/analysis to give most probable hour of the day for the earthquakes bucketed by the range of magnitude (0-1,1-2,2-3,3-4,4-5,5-6,>6   For border #values in the bucket, include them in the bucket where the value is a lower limit so for 1 include it in 1-2 bucket)
#######################################################################################################

#结论： 在早上10：00 之间发生 0 ~ 1 级的地震是最多的。总共发生了4557次。

In [13]:
%sql

select
  inn.Time_In_Hour,
  inn.mag_level,
  count(inn.mag) as EarthQuakeCount
from
(
select 
       hour(time) as Time_In_Hour,
       mag,
       case when mag < 1 then '0-1'
       when mag >=1 and mag < 2 then '0-1'
       when mag >=2 and mag < 3 then '2-3'
       when mag >=3 and mag < 4 then '3-4'
       when mag >=4 and mag < 5 then '4-5'
       when mag >=5 and mag < 6 then '5-6'
       when mag >=6  then '>6'
       end as mag_level
from df_sql
) inn
group by inn.Time_In_Hour,
        inn.mag_level
order by count(inn.mag) desc



Time_In_Hour,mag_level,EarthQuakeCount
10,0-1,4557
11,0-1,4338
7,0-1,4306
12,0-1,4270
6,0-1,4236
8,0-1,4218
5,0-1,4210
9,0-1,4200
3,0-1,4189
4,0-1,4176


In [14]:
#Any interesting visualization (using any open source product or trial version of any product) for these questions. Just attach screenshot.

In [15]:
%sql
select 
 type,
 count(*)
from df_sql
where type <> 'earthquake'
group by type
order by count(*) desc

--结论： 不是全部的地震都是真的地震，有些还是其他原因造成的，比如说矿场炸矿洞时。

type,count(1)
quarry blast,1148
ice quake,807
explosion,759
other event,325
mining explosion,248
chemical explosion,52
sonic boom,14
landslide,1
collapse,1
nuclear explosion,1


In [16]:
%sql
select 
 hour(time),
 count(*)
from df_sql
where type = 'quarry blast'
group by hour(time)
order by count(*) desc

hour(time),count(1)
19,322
20,202
18,172
17,115
23,84
21,78
22,67
16,50
15,25
0,17


In [17]:
%sql
select 
 hour(time),
 count(*)
from df_sql
where type = 'quarry blast'
group by hour(time)
order by count(*) desc

--结论： 矿场最喜欢在下午17：00 ~ 20：00之间炸矿洞时, roughly 71% 。 I wonder why...

hour(time),count(1)
19,322
20,202
18,172
17,115
23,84
21,78
22,67
16,50
15,25
0,17
